In [7]:
!pip install gradio


[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
!pip install fpdf


[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
!pip install pymupdf

In [12]:
import os
import subprocess
from tkinter import Tk, Button, Label, filedialog
from PIL import Image, ImageTk
from fpdf import FPDF
import fitz
import threading

# Global variables
pdf_path = None
image_paths = []
current_image_index = 0
stop_thread = False  # Flag to stop the background thread

# Function to create a folder for images
def create_output_folder(pdf_path):
    pdf_name = os.path.splitext(os.path.basename(pdf_path))[0]
    output_folder = os.path.join(os.path.dirname(pdf_path), f"{pdf_name}_output_images")
    os.makedirs(output_folder, exist_ok=True)
    return output_folder

# Function to convert PDF pages to images
def pdf_to_images(pdf_path, output_folder, dpi=150):
    images = []
    doc = fitz.open(pdf_path)
    for page_num, page in enumerate(doc):
        pix = page.get_pixmap(dpi=dpi)
        image_path = os.path.join(output_folder, f"page_{page_num + 1}.jpg")
        pix.save(image_path)
        images.append(image_path)
    return images

# Function to compress an image
def compress_image(image_path, quality=85):
    image = Image.open(image_path)
    compressed_image_path = image_path.replace(".jpg", "_compressed.jpg")
    image.save(compressed_image_path, "JPEG", quality=quality)
    return compressed_image_path

# Function to open a PDF file
def open_pdf():
    global pdf_path, image_paths
    pdf_path = filedialog.askopenfilename(filetypes=[("PDF Files", "*.pdf")])
    output_folder = create_output_folder(pdf_path)
    if pdf_path:
        image_paths = pdf_to_images(pdf_path, output_folder)
        show_image()
        lbl_info.config(text="New PDF conversion to images")

# Function to display the current image
def show_image():
    global current_image_index
    if image_paths:
        image_path = image_paths[current_image_index]
        image = Image.open(image_path)
        image = image.resize((400, 400))
        photo = ImageTk.PhotoImage(image)
        lbl_image.config(image=photo)
        lbl_image.image = photo

# Function to edit the current image using MS Paint
def edit_image():
    global current_image_index
    if current_image_index < len(image_paths):
        image_path = image_paths[current_image_index]
        subprocess.Popen(['mspaint.exe', image_path])
        root.after(2000, show_image)

# Function to save the changes and move to the next image
def save_and_next():
    global current_image_index
    current_image_index += 1
    n = len(image_paths)
    if current_image_index < n:
        if current_image_index == n - 1:
            lbl_info.config(text="Last Editable image")
        show_image()
        edit_image()
    else:
        lbl_info.config(text="No more images to edit")

# Function to convert edited images back into a PDF
def convert_to_pdf():
    global image_paths, stop_thread
    lbl_info.config(text="Converting to PDF...")
    stop_thread = False  # Reset stop flag
    pdf_name = os.path.basename(pdf_path)
    output_pdf = os.path.join(os.path.dirname(pdf_path), f"revised_{pdf_name}")
    thread = threading.Thread(target=images_to_pdf, args=(image_paths, output_pdf), daemon=True)
    thread.start()
    root.protocol("WM_DELETE_WINDOW", lambda: on_closing(thread))

# Function to convert images into a PDF
def images_to_pdf(image_paths, output_pdf):
    global stop_thread
    pdf = FPDF()
    compressed_image_paths = []
    for image_path in image_paths:
        if stop_thread:  # Check if the stop flag is set
            break
        compressed_image_path = compress_image(image_path)
        compressed_image_paths.append(compressed_image_path)
        pdf.add_page()
        pdf.image(compressed_image_path, 0, 0, 210, 297)  # Adjust image size if needed

    if not stop_thread:
        pdf.output(output_pdf, "F")
        lbl_info.config(text=f"Revised PDF saved as: {output_pdf}")
        # Clean up compressed images
        for compressed_image_path in compressed_image_paths:
            os.remove(compressed_image_path)
    else:
        lbl_info.config(text="Conversion stopped")

# Function to handle window close event
def on_closing(thread):
    global stop_thread
    stop_thread = True  # Set the stop flag to True
    thread.join(1.0)  # Wait for the thread to finish
    root.destroy()  # Close the GUI window
    os._exit(0)  # Forcefully exit the program

# GUI setup
root = Tk()
root.title("PDF Image Editor")

# Image Label
lbl_image = Label(root)
lbl_image.pack()

# Info Label
lbl_info = Label(root, text="")
lbl_info.pack()

# Buttons
btn_open_pdf = Button(root, text="Open PDF", command=open_pdf)
btn_open_pdf.pack()

btn_edit_image = Button(root, text="Edit Image", command=edit_image)
btn_edit_image.pack()

btn_save_and_next = Button(root, text="Save & Next", command=save_and_next)
btn_save_and_next.pack()

btn_convert_pdf = Button(root, text="Convert to PDF", command=convert_to_pdf)
btn_convert_pdf.pack()

root.mainloop()

: 